# Main Document to Load and Traverse Data

> Expected Data: school admissions preferences, number of students admitted per attribute category, etc.

In [15]:
import pandas as pd
import numpy as np
import os

cwd = os.getcwd()
testing = True

## Schema:
Nutrition Label:
- School Label:
  - Name
  - ID
  - AP courses offered
  - Location
  - Subject Specialization
  - Accomodations
  - Admission Preferences
   - gpa cutoff
   - zone
   - subject specification


- Student Label:
  - Name
  - Location
  - Zone
  - School ID
  - Subject preference
  - AP course preferences

## Task 1: Load student demographics and preferences and join

In [6]:
student_df = pd.read_csv(cwd + "/Data/student_info_with_demographics.csv")
student_df

,Student_Id,Residential_District,swd,poverty,sex,ell,ENI,Black,Hispanic,Multi-Racial,White,Asian,Native American,Missing Race/Ethnicity Data,school,Math_score,ELA_score
0,student_36332,Residential District Unknown,0,0,1,0,0.723,0,1,0,0,0,0,0,30Q127,2.000000,3.125000
1,student_36144,Residential District Unknown,0,1,0,0,0.684,0,1,0,0,0,0,0,27Q137,2.153846,2.750000
2,student_37038,Residential District Unknown,1,1,1,0,0.881,0,1,0,0,0,0,0,24Q061,2.846154,1.753425
3,student_614,Residential District Unknown,0,0,1,0,0.191,0,0,0,1,0,0,0,02M114,4.275862,4.181818
4,student_21981,Residential District Unknown,1,0,1,0,0.304,0,0,0,1,0,0,0,02M312,4.017241,3.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71245,student_22462,Residential District 01,0,0,1,0,0.650,0,1,0,0,0,0,0,01M450,2.000000,1.972603
71246,student_30857,Residential District 01,1,0,1,0,0.621,0,0,0,0,1,0,0,01M184,3.000000,3.125000
71247,student_57833,Residential District 01,0,0,0,0,0.256,1,0,0,0,0,0,0,01M539,2.615385,4.030303
71248,student_32259,Residential District 01,0,1,0,0,0.950,0,0,0,0,1,0,0,01M188,3.083333,2.000000


In [8]:
ranking_df = pd.read_csv(cwd + "/Data/student_prefs.csv")
ranking_df

,Student_Id,School,Rank,Rating
0,student_68963,02M411,0,1.000000
1,student_68963,02M376,1,0.500000
2,student_68963,02M316,2,0.333333
3,student_68963,02M438,3,0.250000
4,student_68963,01M448,4,0.200000
...,...,...,...,...
492363,student_36332,02M414,2,0.333333
492364,student_36332,20K490,3,0.250000
492365,student_36332,18K637,4,0.200000
492366,student_36332,28Q686,5,0.166667


In [16]:
student_ids = list(student_df["Student_Id"].unique())
if testing:  # subset of data for testing
    student_ids = student_ids[:100]

student_rankings = {}
for sid in student_ids:
    temp = ranking_df[ranking_df["Student_Id"]==sid]
    temp.sort_values("Rank", inplace=True, ascending=True)
    student_rankings[sid] = list(temp["School"])

student_rankings

/var/folders/mv/k0krwyts3z5cfn3y2tltw0sc0000gn/T/ipykernel_19089/4210586680.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.sort_values("Rank", inplace=True, ascending=True)


{'student_36332': ['22K405',
  '28Q440',
  '02M414',
  '20K490',
  '18K637',
  '28Q686',
  '24Q585'],
 'student_36144': ['20K490',
  '22K405',
  '03M479',
  '01M539',
  '24Q585',
  '26Q495',
  '02M412',
  '02M529',
  '02M519',
  '24Q610',
  '03M417'],
 'student_37038': ['15K497', '02M416', '02M418', '17K122', '30Q575', '22K535'],
 'student_614': ['27Q475', '21K525', '07X670', '02M418', '15K656', '02M630'],
 'student_21981': ['24Q600',
  '24Q550',
  '03M479',
  '13K419',
  '29Q272',
  '02M533',
  '02M413',
  '02M260'],
 'student_59890': ['20K485',
  '02M439',
  '03M541',
  '26Q430',
  '30Q286',
  '02M529',
  '02M414',
  '32K554',
  '28Q620'],
 'student_50244': ['22K425',
  '20K505',
  '22K405',
  '02M414',
  '02M316',
  '13K670',
  '02M411'],
 'student_57724': ['02M260',
  '28Q328',
  '24Q610',
  '21K525',
  '02M413',
  '02M500',
  '02M418',
  '02M408',
  '06M467',
  '21K348'],
 'student_12142': ['02M400', '11X544', '02M416', '02M413', '24Q550'],
 'student_38624': ['27Q650',
  '04M435',

In [17]:
# saving student rankings
np.save(cwd+"/Data/student_rankings.npy", student_rankings)
print("saved at", "/Data/student_rankings.npy")

saved at /Data/student_rankings.npy


## Task 2: Aggregate School Codes & Generate Random Rankings

### Subtask 2.1: Export

#### INTENDED OUTPUT
student_rankings = {student_id : [school_ids_ranked]}

school_rankings = {school ids : [student_ids_ranked]}

[student_rankings, school_rankings]